<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0


    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-24 14:08:59
-------------------
qualified stocks: 88
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  54.95 K
Current:  1.42 C
-------------------
Today PnL: -36.27 K (-0.26%)
Current PnL: -19.88 L (-13.24%)
CY Booked + Current PnL: -7.78 L (-5.18%)
-------------------
Total profit:  2.17 L
Total loss:  -22.04 L
-------------------
Total Booked + Current PnL: 19.20 L (15.6%)
Total Booked PnL: 39.08 L (31.74%)
Curr Year Booked PnL: 12.10 L (8.53%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.06 L (61.4%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.31%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,VOLTAS,1530.0,3.26,61.0,X-MC,2.81,216705.0,24963.0,12786.0,0.94,13.02,5.90,19.69,99.0,1.95,1.53,20.70,XY25,NTT,AC
58,RELIANCE,1533.0,-9.53,66.0,X-LC,3.17,226044.0,15678.0,13111.0,0.04,7.45,5.80,13.68,37.0,1.20,1.60,25.21,XY25,NTT,REFINERIES
2,ABBOTINDIA,35195.0,-9.24,43.0,X-MC,3.32,89730.0,-948.0,15855.0,-0.68,-1.05,17.67,16.44,101.0,-0.06,0.64,18.92,X40,ATH,PHARMA
78,TTKPRESTIG,770.0,96.16,49.0,M-SC,1.97,84502.0,-16275.0,16368.0,0.50,-16.15,19.37,0.09,245.0,-0.99,0.60,11.02,OX40N,NTT,DURABLES
42,ITC,452.0,-37.08,64.0,X-LC,1.96,203571.0,3433.0,17466.0,0.08,1.72,8.58,10.44,4.0,0.20,1.44,7.37,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-197.73,81.0,M-SC,24.63,169740.0,20520.0,123656.0,8.14,13.75,72.85,96.62,208.0,0.17,1.20,83.70,XY25,NTT,FINANCE
70,SURYODAY,240.00,71.05,64.0,H-SC,12.27,156060.0,-23011.0,88736.0,2.82,-12.85,56.86,36.71,135.0,-0.26,1.10,54.84,XR,NTT,BANKS
82,VAIBHAVGBL,521.00,55.92,62.0,H-SC,4.37,135360.0,-47415.0,158479.0,2.47,-25.94,117.08,60.77,125.0,-0.30,0.96,21.24,XR,NTT,JEWELLERY
36,IEX,219.00,-30.16,62.0,H-SC,17.42,199465.0,-489.0,97718.0,1.71,-0.24,48.99,48.63,115.0,-0.01,1.41,11.28,XR,NTT,MISC
32,HINDZINC,730.22,30.43,54.0,M-LC,11.02,212476.0,7400.0,105176.0,1.31,3.61,49.50,54.89,52.0,0.07,1.50,28.57,X5K,ATH,METALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDUNILVR,2922.00,-10.50,43.0,X-LC,7.66,251150.0,-5913.0,41038.0,-3.46,-2.30,16.34,13.67,24.0,-0.14,1.78,17.64,XY25,NTT,FMCG
26,GREENPANEL,537.00,225.58,51.0,M-SC,5.30,145424.0,-37654.0,116630.0,-3.12,-20.57,80.20,43.14,230.0,-0.32,1.03,36.79,XY24,NTT,MISC
7,ASIANTILES,137.00,7055.56,62.0,L-SC,7.19,78521.0,-15289.0,91909.0,-2.62,-16.30,117.05,81.67,269.0,-0.17,0.56,52.06,XR,NTT,CERAMICS
21,COLPAL,3767.14,-4.54,45.0,X-MC,8.16,223000.0,-40365.0,153714.0,-2.49,-15.33,68.93,43.04,84.0,-0.26,1.58,3.53,XY25,ATH,FMCG
83,VALIANTORG,838.00,-286.53,40.0,H-SC,7.88,103751.0,-59854.0,181170.0,-2.32,-36.58,174.62,74.15,139.0,-0.33,0.73,32.99,XR,NTT,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.0,41.94,48.0,M-MC,10.01,223946.0,-1016.0,168676.0,0.55,-0.45,75.32,74.53,192.0,-0.01,1.59,31.66,XY24,BTT,STEEL
36,IEX,219.0,-30.16,62.0,H-SC,17.42,199465.0,-489.0,97718.0,1.71,-0.24,48.99,48.63,115.0,-0.01,1.41,11.28,XR,NTT,MISC
56,RECLTD,446.0,45.36,46.0,M-LC,5.71,204792.0,1622.0,40508.0,-0.68,0.80,19.78,20.74,55.0,0.04,1.45,6.39,XY25,NTT,FINANCE
87,WIPRO,420.0,-14.25,46.0,M-LC,5.95,150350.0,-595.0,110056.0,-0.74,-0.39,73.20,72.51,53.0,-0.01,1.06,5.60,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,122.87,69.0,H-SC,15.26,140508.0,-1971.0,21498.0,0.27,-1.38,15.30,13.70,163.0,-0.09,0.99,65.84,OX40N,NTT,BANKS
45,KANSAINER,340.0,-66.87,58.0,H-SC,2.37,227934.0,-41733.0,78067.0,-0.85,-15.48,34.25,13.47,138.0,-0.53,1.61,15.90,XY24,NTT,PAINTS
78,TTKPRESTIG,770.0,96.16,49.0,M-SC,1.97,84502.0,-16275.0,16368.0,0.50,-16.15,19.37,0.09,245.0,-0.99,0.60,11.02,OX40N,NTT,DURABLES
73,TATAELXSI,9161.0,-22.20,55.0,H-MC,7.49,105174.0,-26661.0,68889.0,1.18,-20.22,65.50,32.03,98.0,-0.39,0.74,17.80,OX40N,NTT,IT
18,CERA,9475.0,-23.63,44.0,H-SC,2.62,139368.0,-36535.0,78562.0,0.62,-20.77,56.37,23.89,149.0,-0.47,0.99,20.36,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.00,45.36,46.0,M-LC,5.71,204792.0,1622.0,40508.0,-0.68,0.80,19.78,20.74,55.0,0.04,1.45,6.39,XY25,NTT,FINANCE
32,HINDZINC,730.22,30.43,54.0,M-LC,11.02,212476.0,7400.0,105176.0,1.31,3.61,49.50,54.89,52.0,0.07,1.50,28.57,X5K,ATH,METALS
46,KPIGREEN,731.05,6.72,66.0,H-SC,5.18,127336.0,2039.0,56894.0,-0.65,1.63,44.68,47.03,141.0,0.04,0.90,59.44,MH,ATH,POWER
80,UNIONBANK,163.00,-11.63,57.0,M-LC,9.08,160817.0,19977.0,24509.0,-0.99,14.18,15.24,31.59,66.0,0.82,1.14,44.43,XY24,NTT,BANKS
12,BANKINDIA,190.00,-20.50,67.0,H-MC,14.24,205338.0,25530.0,86879.0,-1.49,14.20,42.31,62.52,88.0,0.29,1.45,53.13,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-20.50,67.0,H-MC,14.24,205338.0,25530.0,86879.0,-1.49,14.20,42.31,62.52,88.0,0.29,1.45,53.13,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-39.28,62.0,M-SC,3.87,102600.0,11102.0,67654.0,-2.24,12.13,65.94,86.07,223.0,0.16,0.73,48.99,XR,NTT,DURABLES
32,HINDZINC,730.22,30.43,54.0,M-LC,11.02,212476.0,7400.0,105176.0,1.31,3.61,49.50,54.89,52.0,0.07,1.50,28.57,X5K,ATH,METALS
46,KPIGREEN,731.05,6.72,66.0,H-SC,5.18,127336.0,2039.0,56894.0,-0.65,1.63,44.68,47.03,141.0,0.04,0.90,59.44,MH,ATH,POWER
37,INDIAMART,4810.62,-55.20,44.0,H-SC,7.16,125186.0,1850.0,129780.0,0.52,1.50,103.67,106.72,119.0,0.01,0.89,25.23,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,16.0,X-LC,13.41,147305.0,-129218.0,239282.0,0.98,-46.73,162.44,39.81,54.0,-0.54,1.04,3.82,XY24,NTT,AUTO
24,DMART,5391.45,-17.48,32.0,X-LC,10.15,100937.0,-1649.0,28454.0,-1.46,-1.61,28.19,26.13,38.0,-0.06,0.71,23.55,X40N,ATH,FMCG
53,QUESS,986.00,-48.76,34.0,X-SC,38.77,51104.0,-13902.0,164831.0,-0.70,-21.39,322.54,232.18,198.0,-0.08,0.36,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-27.54,39.0,X-MC,6.80,81810.0,-1700.0,21729.0,-1.34,-2.04,26.56,23.98,82.0,-0.08,0.58,3.67,X40,ATH,APPARELS
52,PGHH,17907.65,-29.62,41.0,X-MC,3.22,203625.0,2805.0,64997.0,-0.95,1.40,31.92,33.76,80.0,0.04,1.44,6.67,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.50,-2.18,49.0,H-LC,0.79,157120.0,-28975.0,76455.0,-0.24,-15.57,48.66,25.51,15.0,-0.38,1.11,15.79,AR,ATH,ELECTRICAL
42,ITC,452.00,-37.08,64.0,X-LC,1.96,203571.0,3433.0,17466.0,0.08,1.72,8.58,10.44,4.0,0.20,1.44,7.37,X40,NTT,FMCG
85,VOLTAS,1530.00,3.26,61.0,X-MC,2.81,216705.0,24963.0,12786.0,0.94,13.02,5.90,19.69,99.0,1.95,1.53,20.70,XY25,NTT,AC
29,HAVELLS,2069.16,-5.47,49.0,X-MC,2.88,234374.0,-13357.0,88406.0,0.40,-5.39,37.72,30.30,92.0,-0.15,1.66,7.18,X40,ATH,ELECTRICAL
58,RELIANCE,1533.00,-9.53,66.0,X-LC,3.17,226044.0,15678.0,13111.0,0.04,7.45,5.80,13.68,37.0,1.20,1.60,25.21,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.76,34.0,X-SC,38.77,51104.0,-13902.0,164831.0,-0.70,-21.39,322.54,232.18,198.0,-0.08,0.36,0.00,XY24,NTT,MISC
21,COLPAL,3767.14,-4.54,45.0,X-MC,8.16,223000.0,-40365.0,153714.0,-2.49,-15.33,68.93,43.04,84.0,-0.26,1.58,3.53,XY25,ATH,FMCG
51,PAGEIND,51769.93,-27.54,39.0,X-MC,6.80,81810.0,-1700.0,21729.0,-1.34,-2.04,26.56,23.98,82.0,-0.08,0.58,3.67,X40,ATH,APPARELS
74,TATAMOTORS,1065.00,-53.17,16.0,X-LC,13.41,147305.0,-129218.0,239282.0,0.98,-46.73,162.44,39.81,54.0,-0.54,1.04,3.82,XY24,NTT,AUTO
3,ACC,3906.00,-37.64,50.0,X-MC,2.90,185230.0,-52721.0,205365.0,-0.18,-22.16,110.87,64.15,174.0,-0.26,1.31,3.90,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.76,34.0,X-SC,38.77,51104.0,-13902.0,164831.0,-0.70,-21.39,322.54,232.18,198.0,-0.08,0.36,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-27.54,39.0,X-MC,6.80,81810.0,-1700.0,21729.0,-1.34,-2.04,26.56,23.98,82.0,-0.08,0.58,3.67,X40,ATH,APPARELS
57,RELAXO,1176.00,-39.30,45.0,X-SC,4.68,82739.0,-62421.0,138348.0,-0.82,-43.00,167.21,52.31,136.0,-0.45,0.59,10.75,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.24,43.0,X-MC,3.32,89730.0,-948.0,15855.0,-0.68,-1.05,17.67,16.44,101.0,-0.06,0.64,18.92,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-35.44,46.0,X-SC,4.45,96463.0,-32207.0,77508.0,-0.46,-25.03,80.35,35.20,219.0,-0.42,0.68,10.72,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-27.72,56.0,X-LC,12.36,287715.0,-58241.0,124955.0,-0.40,-16.83,43.43,19.28,1.0,-0.47,2.04,6.36,X40,ATH,IT
40,INFY,2275.00,-17.09,57.0,X-LC,7.19,323654.0,10766.0,160921.0,-0.59,3.44,49.72,54.87,3.0,0.07,2.29,10.32,X40,BTT,IT
42,ITC,452.00,-37.08,64.0,X-LC,1.96,203571.0,3433.0,17466.0,0.08,1.72,8.58,10.44,4.0,0.20,1.44,7.37,X40,NTT,FMCG
84,VBL,671.64,-18.39,52.0,X-LC,9.88,293703.0,-22139.0,134810.0,-1.12,-7.01,45.90,35.67,5.0,-0.16,2.08,5.87,X40N,ATH,FMCG
1,ABB,7934.00,-40.35,49.0,H-LC,6.31,249360.0,-12259.0,131463.0,0.26,-4.69,52.72,45.57,7.0,-0.09,1.77,6.10,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7055.56,62.0,L-SC,7.19,78521.0,-15289.0,91909.0,-2.62,-16.30,117.05,81.67,269.0,-0.17,0.56,52.06,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,26.97,80986.0,-32563.0,72612.0,0.24,-28.68,89.66,35.27,268.0,-0.45,0.57,98.21,XR,NTT,HOTELS
49,MASFIN,398.61,-19.60,48.0,H-SC,9.55,91260.0,-6720.0,28327.0,-0.94,-6.86,31.04,22.05,152.0,-0.24,0.65,32.65,XR,ATH,FINANCE
86,WHIRLPOOL,2270.00,-39.28,62.0,M-SC,3.87,102600.0,11102.0,67654.0,-2.24,12.13,65.94,86.07,223.0,0.16,0.73,48.99,XR,NTT,DURABLES
46,KPIGREEN,731.05,6.72,66.0,H-SC,5.18,127336.0,2039.0,56894.0,-0.65,1.63,44.68,47.03,141.0,0.04,0.90,59.44,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-197.73,81.0,M-SC,24.63,169740.0,20520.0,123656.0,8.14,13.75,72.85,96.62,208.0,0.17,1.20,83.70,XY25,NTT,FINANCE
79,UJJIVANSFB,60.00,122.87,69.0,H-SC,15.26,140508.0,-1971.0,21498.0,0.27,-1.38,15.30,13.70,163.0,-0.09,0.99,65.84,OX40N,NTT,BANKS
68,SONACOMS,806.63,-32.99,69.0,M-SC,9.62,83396.0,-17763.0,57760.0,-0.31,-17.56,69.26,39.54,202.0,-0.31,0.59,17.60,AR,ATH,AUTO
5,ANGELONE,3033.00,16.33,70.0,X-SC,6.39,205574.0,14568.0,43129.0,-0.16,7.63,20.98,30.21,157.0,0.34,1.46,30.10,X40N,NTT,FINANCE
59,REPCOHOME,880.00,-55.29,72.0,H-SC,2.97,257830.0,-28675.0,286011.0,1.19,-10.01,110.93,89.82,134.0,-0.10,1.83,36.41,XY24,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.43
1,25,43.83
2,50,74.77


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.48
LC    32.68
MC    22.84
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.29
X40      16.02
XY25     12.29
XR       11.29
X40N      9.31
AR        7.92
OX40N     7.63
X200      1.76
X5K       1.50
SR        1.09
MH        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.64
X-LC    21.86
X-MC    17.10
M-SC    12.74
M-LC     5.15
H-LC     4.64
X-SC     4.63
H-MC     3.85
M-MC     1.59
L-SC     1.47
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.67,-4.02,37.97
IT,12.71,-17.12,80.27
FINANCE,12.24,-9.43,59.03
MISC,6.93,-17.43,82.50
BANKS,6.60,-6.84,63.05
PAINTS,5.64,-16.09,33.39
ELECTRICAL,5.31,-9.74,48.24
INSURANCE,3.88,-0.79,38.36
AUTO,2.80,-44.99,105.50


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3296226.0,22
XR,1313759.0,14
AR,1178074.0,9
X40,782177.0,12
X40N,614454.0,8
OX40N,569466.0,10
XY25,446612.0,8
SR,267746.0,2
X5K,105176.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3379414.0,25
M-SC,1208338.0,15
X-MC,1120327.0,13
X-LC,1051352.0,13
X-SC,526715.0,6
H-MC,335448.0,3
H-LC,283127.0,3
M-LC,280249.0,4
L-SC,253872.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1168672.0      6
           AR         845884.0      5
           XR         776731.0      7
M-SC       XY24       696752.0      6
X-MC       XY24       584083.0      4
X-LC       X40        451815.0      5
           XY24       309023.0      2
X-SC       X40N       284376.0      4
H-SC       SR         267746.0      2
           OX40N      263487.0      4
X-MC       X40        252854.0      6
X-LC       X40N       213188.0      3
H-LC       AR         207918.0      2
H-MC       XY24       179680.0      1
M-MC       XY24       168676.0      1
X-MC       XY25       166500.0      2
X-SC       XY24       164831.0      1
L-SC       XR         164521.0      2
M-SC       OX40N      147739.0      4
           AR         124272.0      2
           XY25       123656.0      1
X-MC       X40N       116890.0      1
M-SC       XR         115919.0      2
M-LC       XR         110056.0      1
           X5K        105176.0      1
L-SC       OX40N       89351.0      1
H-MC       XR          86879.0      1
X-SC       X40         77508.0      1
X-LC       XY25        77326.0      3
H-LC       X200        75209.0      1
H-MC       OX40N       68889.0      1
L-MC       XR          59653.0      1
H-SC       MH          56894.0      1
M-LC       XY25        40508.0      1
L-LC       XY25        38622.0      1
M-LC       XY24        24509.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
